In [6]:
pip install PyMuPDF

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

  Obtaining dependency information for PyMuPDF from https://files.pythonhosted.org/packages/2b/14/b3d950d252e64997a28bc2452d1fa19161888c80ee49cacee9bb06a9ec05/PyMuPDF-1.23.6-cp311-none-win_amd64.whl.metadata
  Obtaining dependency information for PyMuPDFb==1.23.6 from https://files.pythonhosted.org/packages/f4/f6/487aba0931328119d8674637dc52b72d2f3b75ab373abd223436bb4bf605/PyMuPDFb-1.23.6-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.5 MB 435.7 kB/s eta 0:00:08
    --------------------------------------- 0.1/3.5 MB 653.6 kB/s eta 0:00:06
   ---- ----------------------------------- 0.4/3.5 MB 2.9 MB/s eta 0:00:02
   ------ --------------------------------- 0.6/3.5 MB 3.8 MB/

In [19]:
pip install -U sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/86.0 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/86.0 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/86.0 kB ? eta -:--:--
     --------------------------- ---------- 61.4/86.0 kB 544.7 kB/s eta 0:00:01
     -------------------------------------- 86.0/86.0 kB 605.7 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/13/24/23cdf7e7dc33e5c01588c315f8424d31afa9edb05a80168f3d44f7178ff7/torchvision-0.16.1-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ----------- ---------------------------- 0.3/1.1 MB 10.6 MB/s eta 0:00:01
   ---------------------- ----------------- 0.6/1.1 MB 8.1 MB/s eta 0:00:01
   

from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(embeddings)

Open the PDF file with the appropriate encoding ('utf-8') when reading it.
Load the PDF text into the pdf_text variable.
Split the pdf_text into chunks using the split() method.
Iterate through the chunks and print the embeddings for each chunk.
Make sure you have the RAGS library installed (pip install rags) and that you have the correct path to your PDF file. Additionally, note that this example assumes that RAGS provides a function called load_model() for loading the word embedding model. If there are any changes in the library's API, you may need to adjust the code accordingly.

len(docs)

In [7]:
pip install pinecone-client

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Initiate the pinecone client
import os
import pinecone

pinecone_api = os.getenv("PINECONE_API")
pinecone_env = os.getenv("PINECONE_ENV")

pinecone.init(
    api_key = pinecone_api,
    environment = pinecone_env
)

C:\Users\vl\AppData\Roaming\Python\Python311\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
index_name = "pdfsearchdx"
pinecone.create_index(index_name, dimension=384, metric="euclidean")
pinecone.describe_index(index_name)
index = pinecone.Index(index_name)

jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

In [3]:
import fitz  # PyMuPDF
from sentence_transformers import SentenceTransformer
import pinecone
import numpy as np

index_name = "pdfsearchdx"
index = pinecone.Index(index_name)

path_to_pdf = 'YL-CV-masked.pdf' 

# Extract text from the PDF using PyMuPDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(doc.page_count):
        page = doc[page_num]
        text += page.get_text()
    doc.close()
    return text

def embed(data):#https://huggingface.co/sentence-transformers
    model = SentenceTransformer('all-MiniLM-L6-v2')#https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
    embeddings = model.encode(data).tolist()
    return embeddings

def remove_non_ascii(s):
    return "".join(c for c in s if ord(c)<128)

# Load the PDF text
pdf_text = extract_text_from_pdf(path_to_pdf)
#print(pdf_text)

# Split the text into chunks
chunks = pdf_text.split(".")

# Process each chunk and print the embeddings
sentences = []
#metadatas = []
for chunk in chunks:
    sentences.append(remove_non_ascii(chunk))
    #metadatas.append(remove_non_ascii(chunk))
    
embeddings = embed(sentences)
print(len(sentences))
print(len(embeddings))
#print(len(metadatas))
#print(sentences)
#print(embeddings)
#print(metadatas)
index.upsert(vectors=zip(sentences, embeddings))
    

53
53


{'upserted_count': 53}

In [4]:
from sentence_transformers import SentenceTransformer
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = str(os.getenv("HUGGINGFACEHUB_API_TOKEN"))

query_questions = ["What is my blog?",]

model = SentenceTransformer('all-MiniLM-L6-v2')
query_vectors = [model.encode(str(question)).tolist() for question in query_questions]

query_results = index.query(vector=query_vectors, top_k=2)
print(query_results)

{'matches': [{'id': 'com \nIdeas Blog: \nhttps://edisonideas',
              'score': 0.923787475,
              'values': []},
             {'id': 'wordpress', 'score': 1.01926112, 'values': []}],
 'namespace': ''}
